In [1]:

# Import necessary libraries for data manipulation, plotting, and network analysis
import warnings
warnings.filterwarnings("ignore")

import pandapower as pp
import simbench as sb
import pandapower.networks as nw

import GridCalEngine.api as gce  # For interfacing with the GridCal API
from GridCalEngine.IO.file_handler import FileOpen, FileSave


import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import GC_utils
import GC_PandaPowerImporter
from GC_DistributionNetworkReconfiguration import DistributionNetworkReconfiguration


import logging  # For logging messages
logging.basicConfig(
    level=logging.ERROR,  # Set the log level to DEBUG
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Set the log format
    datefmt='%Y-%m-%d %H:%M:%S'  # Set the date format
)

In [2]:
#This function call DNR library, executes the selected DNR method and prints the results, to easy the comparison

def executeTest(dnr, method, TieLinesID=[], randomMST=False, one=False, current_power=True, algorithm="prim", NumCandidates=10, fitness_ratio=1, loss_factor=0.02, MutationProbability=0.4, PopulationSize=16, Niter=20, ElitePopulation=2):
    if method == None:
        _, losses = GC_utils.GC_PowerFlow(dnr.grid, config=TieLinesID)
        radiality = GC_utils.CheckRadialConnectedNetwork(dnr.grid)
        print("losses:", losses)
        print("radiality:", radiality)
        print("Loops:")
        for loop in GC_utils.SearchLoopsLines(dnr.grid):
            print(GC_utils.GC_Line_idtag2name_array(dnr.grid,loop))
    else:      
#       try:
        dnr.NumPF=0
        DisabledLinesID = dnr.Solve(method=method, TieLines=TieLinesID, randomMST=randomMST, one=one, current_power=current_power, algorithm=algorithm, NumCandidates=NumCandidates, fitness_ratio=fitness_ratio, loss_factor=loss_factor, MutationProbability=MutationProbability, PopulationSize=PopulationSize, Niter=Niter, ElitePopulation=ElitePopulation)

        _, losses = GC_utils.GC_PowerFlow(dnr.grid, config=DisabledLinesID)
        radiality = GC_utils.CheckRadialConnectedNetwork(dnr.grid)
        print("DNR solution by ", method, ":", GC_utils.GC_Line_idtag2name_array(dnr.grid, DisabledLinesID))
        print("Number of executed Power Flows:", dnr.NumPF)
        print("losses:", losses)
        print("radiality:", radiality)
        print("Loops:")
        for loop in GC_utils.SearchLoopsLines(dnr.grid):
            print(GC_utils.GC_Line_idtag2name_array(dnr.grid,loop))
#        except:
#            print(method, " method failed")        


In [3]:
print('Algorithms to find the optimal distribution network configuration')


Algorithms to find the optimal distribution network configuration


In [4]:
gridGC = FileOpen("D:\\15_Thesis-code\\02_DistributionNetworkOperationFramework\\03_NetworkExamples\\gridcal\\case14.m").open()
TieLinesName=['6_12_1', '13_14_1', '2_4_1','4_5_1','4_7_1','2_3_1','1_2_1']
TieLinesID = GC_utils.GC_Line_Name2idtag_array(gridGC, TieLinesName)
dnr = DistributionNetworkReconfiguration(gridGC, verbose_logging=logging.ERROR)

<img src="14buses.png" width="300" height="300">


In [5]:
GC_utils.NetworkReconfiguration(gridGC, all=True, value_all=True)
print(f"The 14 buses cases is loaded and with all lines connected it has {len(GC_utils.SearchLoopsLines(gridGC))} loops")
GC_utils.NetworkReconfiguration(gridGC, all=True, selected_configuration=TieLinesID, value_all=True, value_configuration=False)
print(f"The 14 buses cases is loaded and with tie lines disconnected it has {len(GC_utils.SearchLoopsLines(gridGC))} loops")

The 14 buses cases is loaded and with all lines connected it has 7 loops
The 14 buses cases is loaded and with tie lines disconnected it has 0 loops


In [6]:
## executes the power flow and radiality check for the original network with all lines active
executeTest(dnr, None, None)  

losses: 115.22919336533134
radiality: (True, True, True)
Loops:


In [7]:
res, losses = GC_utils.GC_PowerFlow(dnr.grid)
print(res.results.get_bus_df().Vm.max())
print(res.results.get_bus_df().Vm.min())

1.09
0.8150326689473189


In [8]:
## executes the power flow and radiality check for the original network with a random set of lines deactivated to achieve a radial network
executeTest(dnr, None, TieLinesID)  

losses: 115.22919336533134
radiality: (True, True, True)
Loops:


In [9]:
## executes the power flow and radiality check for the original network with an improved set of lines deactivated to achieve a radial network
BetterTieLinesName = ['2_5_1', '3_4_1', '4_5_1', '4_7_1', '4_9_1', '10_11_1', '12_13_1']
BetterTieLinesID = GC_utils.GC_Line_Name2idtag_array(gridGC, BetterTieLinesName)
executeTest(dnr, None, BetterTieLinesID)  

losses: 28.9164418707725
radiality: (True, True, True)
Loops:


In [10]:
res, losses = GC_utils.GC_PowerFlow(dnr.grid)
print(res.results.get_bus_df().Vm.max())
print(res.results.get_bus_df().Vm.min())

1.0900007814035269
0.9583563381962484


In [11]:
#Executes the DNR algorithm by using the Merlin method to find the optimal configuration of the distribution network
executeTest(dnr, 'Merlin', TieLinesID)

DNR solution by  Merlin : ['2_5_1', '3_4_1', '4_5_1', '4_9_1', '9_14_1', '10_11_1', '12_13_1']
Number of executed Power Flows: 12
losses: 16.939941228665308
radiality: (True, True, True)
Loops:


In [12]:
#Executes the DNR algorithm by using the Baran method to find the optimal configuration of the distribution network
executeTest(dnr, 'Baran', TieLinesID)

DNR solution by  Baran : ['6_13_1', '9_14_1', '2_5_1', '4_9_1', '7_9_1', '2_3_1', '1_2_1']
Number of executed Power Flows: 19
losses: 60.26201684950429
radiality: (True, True, True)
Loops:


In [13]:
#Executes the DNR algorithm by using the Salkuti method to find the optimal configuration of the distribution network
executeTest(dnr, 'Salkuti', TieLinesID)

DNR solution by  Salkuti : ['12_13_1', '9_14_1', '4_9_1', '2_5_1', '4_5_1', '3_4_1', '2_4_1']
Number of executed Power Flows: 19
losses: 45.602350710758216
radiality: (True, True, True)
Loops:


In [14]:
#Executes the DNR algorithm by using the MSTgreedy method to find the optimal configuration of the distribution network
executeTest(dnr=dnr, method='MSTgreedy', TieLinesID=TieLinesID, randomMST=False, one=False, current_power=True, algorithm="prim")

DNR solution by  MSTgreedy : ['2_5_1', '3_4_1', '4_5_1', '4_7_1', '4_9_1', '10_11_1', '12_13_1']
Number of executed Power Flows: 1
losses: 28.9164418707725
radiality: (True, True, True)
Loops:


In [15]:
#Executes the DNR algorithm by using the Khalil method to find the optimal configuration of the distribution network
executeTest(dnr=dnr, method='Khalil', NumCandidates=10, fitness_ratio=1, loss_factor=0.02)

DNR solution by  Khalil : ['6_13_1', '6_11_1', '2_5_1', '4_9_1', '7_9_1', '3_4_1', '1_5_1']
Number of executed Power Flows: 240
losses: 35.09595245426215
radiality: (True, True, True)
Loops:


In [16]:
#Executes the DNR algorithm by using the Jakus method to find the optimal configuration of the distribution network
executeTest(dnr=dnr, method='Jakus', MutationProbability=0.4, PopulationSize=16, Niter=20, ElitePopulation=2, TieLinesID=TieLinesID, fitness_ratio=1, loss_factor=0.02)

DNR solution by  Jakus : ['2_5_1', '3_4_1', '4_5_1', '4_9_1', '6_12_1', '7_9_1', '9_14_1']
Number of executed Power Flows: 298
losses: 19.077012547672588
radiality: (True, True, True)
Loops:


In [17]:
## implementation of Taylor Mathematical Programming method is not working properly and needs to be completely revised
dnr.NumPF=0
disabled_lines = dnr.Solve(method="Taylor", algorithm="QP", solver="ipopt", bigM=10e6, Imax=10, vmin=0.8, vmax=1.2, TieLines=TieLinesID)
_, loss = GC_utils.GC_PowerFlow(gridGC, config=disabled_lines)
radiality = GC_utils.CheckRadialConnectedNetwork(gridGC)
print(f"Taylor: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )
#except:
#    print("falla Taylor")  

Taylor: The new optimal configuration losses:13.347859486581399, radiality:(False, True, False), numPF:0 


In [18]:
## this method takes 41 seconds to execute for the 14 buses case, finding 4351 possible networks to be evaluated
## the optimal radial network has the following lines disabled : ['12_13_1', '3_4_1', '4_7_1', '2_5_1', '9_14_1', '4_5_1', '9_10_1']
## obtaining losses of 16.58 MW
try:
    disabled_lines = dnr.Solve(method="Morton", TieLines=TieLinesID)
    _, loss = GC_utils.GC_PowerFlow(gridGC, config=disabled_lines)
    radiality = GC_utils.CheckRadialConnectedNetwork(gridGC)
    print("Morton: ", GC_utils.GC_Line_idtag2name_array(gridGC,disabled_lines), loss, radiality )   
except:
    print("falla Morton")


[['a4ab7adf9be8431da7cdd065bc0ae1ba', '9635a1b4e4a446fea378726babab1a58', '552753507c9343cebd87b1f79e1a6bb2'], ['725421c2dff74ee2b0f8e0059440536f', '9635a1b4e4a446fea378726babab1a58', 'a73fbea1f68046d8b36c04454479859b', 'b36325c1f9064c1a81978b9e48190f8e', '350a5fc58c4049cebd9a4f333e384fc1', '386d5a12b5354bb58d9fe75ee4c5dd77'], ['689e820cbb9349928c23ce6f3b27af64', '86e64234322d4b079dde044ed497928b', '0a1686dd013a4bed822dbc2052bb6587'], ['0a1686dd013a4bed822dbc2052bb6587', 'c62f7485d52d4b62b8574245451d9ac5', '843102153e00437aa10932da150751bf', '9635a1b4e4a446fea378726babab1a58', 'b36325c1f9064c1a81978b9e48190f8e', '386d5a12b5354bb58d9fe75ee4c5dd77'], ['3df826f5fb714ef39eba30f52bfe60a0', 'c62f7485d52d4b62b8574245451d9ac5', '774e00e5e1ca45c4bae7d9e3d98d3ecf'], ['793587c0e8ea4fcbb7919f1c459b7114', '689e820cbb9349928c23ce6f3b27af64', 'f68134d0e43d441ab3a81dd6f8af9660'], ['9a62d6aa1f83457098fb03535b19eca6', 'a82151c61f7841088e1569ed265e84f6', '86e64234322d4b079dde044ed497928b']]
found 4351 po